In [1]:
# Cargar Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch torchtext transformers


In [3]:
import torch
from torch import nn
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import os

In [4]:

class TextClassifier(nn.Module):
    def __init__(self, num_classes, pretrained_model_name='distilbert-base-uncased'):
        super(TextClassifier, self).__init__()
        self.tokenizer = DistilBertTokenizer.from_pretrained(pretrained_model_name)
        self.bert = DistilBertModel.from_pretrained(pretrained_model_name)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None):
        output = self.bert(input_ids, attention_mask=attention_mask)[0][:, 0, :]  # extract the [CLS] token representation
        logits = self.fc(output)
        return logits


In [5]:
import torch
from torch.utils.data import Dataset  # Añadir esta línea para importar la clase Dataset

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['texto']
        label = self.data.iloc[idx]['carrera']
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt', return_attention_mask=True)
        return {'input_ids': encoding['input_ids'].squeeze(), 'attention_mask': encoding['attention_mask'].squeeze(), 'label': torch.tensor(label)}


In [6]:
# Cargar tu conjunto de datos desde el archivo CSV
dataset_path = './drive/MyDrive/SIS421/ClasificacionTexto/datos_carreras_3.csv'
df = pd.read_csv(dataset_path)

In [7]:
# Dividir el conjunto de datos en entrenamiento y prueba
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
# Inicializar el modelo y el tokenizer
num_classes = 4  # Número de clases en tu problema
model = TextClassifier(num_classes)
tokenizer = model.tokenizer

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [9]:
# Crear instancias de CustomDataset
train_dataset = CustomDataset(train_df, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer)

In [10]:
# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [11]:
for batch in train_dataloader:
    print(batch)
    break  # solo mostramos el primer lote para evitar demasiada salida


{'input_ids': tensor([[  101,  2474, 15270,  ...,     0,     0,     0],
        [  101,  2474,  3188,  ...,     0,     0,     0],
        [  101,  2033,  7861,  ...,     0,     0,     0],
        ...,
        [  101,  2474, 13749,  ...,     0,     0,     0],
        [  101,  9033,  6633,  ...,     0,     0,     0],
        [  101,  4078,  2906,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'label': tensor([2, 0, 0, 0, 2, 2, 3, 3, 2, 1, 2, 1, 2, 3, 2, 1, 0, 0, 2, 1, 3, 0, 1, 3,
        0, 1, 2, 3, 0, 3, 0, 0, 2, 3, 3, 1, 0, 0, 0, 1, 1, 2, 2, 0, 1, 2, 0, 1,
        2, 0, 0, 1, 0, 3, 2, 2, 1, 1, 3, 3, 1, 1, 0, 3])}


In [12]:
def train_model(model, train_dataloader, test_dataloader, num_epochs=12, learning_rate=1e-4, save_every=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        for batch in train_dataloader:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['label'].to(device)
            optimizer.zero_grad()
            logits = model(input_ids, attention_mask=attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

        # Evaluation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch in test_dataloader:
                input_ids, labels = batch['input_ids'].to(device), batch['label'].to(device)
                logits = model(input_ids)
                _, predicted = torch.max(logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")

        # Save model every 'save_every' epochs
        if (epoch + 1) % save_every == 0:
            save_path = f'./drive/MyDrive/SIS421/ClasificacionTexto/model_checkpoint_epoch_{epoch + 1}.pth'
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item(),
                'accuracy': accuracy
            }, save_path)
            print(f'Model saved at epoch {epoch + 1}.')


In [13]:
# Entrenar el modelo con guardado cada 3 epochs
train_model(model, train_dataloader, test_dataloader, num_epochs=30, learning_rate=1e-4, save_every=3)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1/30, Loss: 0.7153, Accuracy: 0.7069
Epoch 2/30, Loss: 0.3259, Accuracy: 0.7521
Epoch 3/30, Loss: 0.3542, Accuracy: 0.7920
Model saved at epoch 3.
Epoch 4/30, Loss: 0.2002, Accuracy: 0.7868
Epoch 5/30, Loss: 0.2486, Accuracy: 0.7962
Epoch 6/30, Loss: 0.3829, Accuracy: 0.7752
Model saved at epoch 6.
Epoch 7/30, Loss: 0.0730, Accuracy: 0.7815
Epoch 8/30, Loss: 0.0855, Accuracy: 0.7658
Epoch 9/30, Loss: 0.1971, Accuracy: 0.8015
Model saved at epoch 9.
Epoch 10/30, Loss: 0.1670, Accuracy: 0.8099
Epoch 11/30, Loss: 0.1163, Accuracy: 0.8067
Epoch 12/30, Loss: 0.0970, Accuracy: 0.8120
Model saved at epoch 12.
Epoch 13/30, Loss: 0.2000, Accuracy: 0.8120
Epoch 14/30, Loss: 0.1018, Accuracy: 0.8109
Epoch 15/30, Loss: 0.0592, Accuracy: 0.7111
Model saved at epoch 15.
Epoch 16/30, Loss: 0.1047, Accuracy: 0.6775
Epoch 17/30, Loss: 0.1836, Accuracy: 0.7532
Epoch 18/30, Loss: 0.0803, Accuracy: 0.7731
Model saved at epoch 18.
Epoch 19/30, Loss: 0.1200, Accuracy: 0.7689
Epoch 20/30, Loss: 0.0725,

In [23]:
# Cargar el modelo
import torch

def load_model(model, optimizer, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    accuracy = checkpoint['accuracy']
    print(f'Model loaded from epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

# Especifica la ruta del archivo del modelo que deseas cargar
model_path_to_load = './drive/MyDrive/SIS421/ClasificacionTexto/model_checkpoint_epoch_30.pth'  # Cambia la ruta según sea necesario

# Crea una nueva instancia del modelo (asegúrate de que tenga la misma arquitectura que el modelo original)
loaded_model = TextClassifier(num_classes)

# Crea una nueva instancia del optimizador (asegúrate de que tenga los mismos parámetros que el modelo original)
loaded_optimizer = torch.optim.AdamW(loaded_model.parameters(), lr=1e-4)  # Ajusta el learning rate según sea necesario

# Carga el modelo y el optimizador desde el archivo
load_model(loaded_model, loaded_optimizer, model_path_to_load)

Model loaded from epoch 30, Loss: 0.1091, Accuracy: 0.7626


In [15]:
# Dar etiquetas a los resultados para mejorar la presentacion
def label_mapping(label):
    mapping = {
        0: 'Ing. En Sistemas',
        1: 'Ing. En Diseño y Animacion Digital',
        2: 'Ing. En Ciencias de la Computación',
        3: 'Ing. En Tecnologías de la Información y Seguridad'
    }
    return mapping[label]

In [16]:

# Función para realizar predicciones con el modelo cargado
def predict_with_loaded_model(model, tokenizer, text):
    # Tokenizar el texto de entrada
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=model.tokenizer.model_max_length, return_tensors='pt', return_attention_mask=True)

    # Obtener predicciones del modelo cargado
    with torch.no_grad():
        logits = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    # Obtener la etiqueta predicha
    _, predicted_class = torch.max(logits, 1)

    return label_mapping(predicted_class.item())

# Ejemplo de texto para probar el modelo cargado
texto_a_probar_cargado = "siempre he estado interesado en la seguridad de la informacion y como se resguardan los datos"

# Probar el modelo con el texto de ejemplo
predicted_label_cargado = predict_with_loaded_model(loaded_model, tokenizer, texto_a_probar_cargado)

# Imprimir la etiqueta predicha de forma legible
print(f"Texto: {texto_a_probar_cargado}")
print(f"Etiqueta Predicha con modelo cargado: {predicted_label_cargado}")

Texto: siempre he estado interesado en la seguridad de la informacion y como se resguardan los datos
Etiqueta Predicha con modelo cargado: Ing. En Tecnologías de la Información y Seguridad


In [17]:
# Ejemplo de texto para probar el modelo cargado
texto_a_probar_cargado = "siempre he estado interesado en ingenieria social"

# Probar el modelo con el texto de ejemplo
predicted_label_cargado = predict_with_loaded_model(loaded_model, tokenizer, texto_a_probar_cargado)

# Imprimir la etiqueta predicha de forma legible
print(f"Texto: {texto_a_probar_cargado}")
print(f"Etiqueta Predicha con modelo cargado: {predicted_label_cargado}")

Texto: siempre he estado interesado en ingenieria social
Etiqueta Predicha con modelo cargado: Ing. En Tecnologías de la Información y Seguridad


In [18]:
# Ejemplo de texto para probar el modelo cargado
texto_a_probar_cargado = "Adoro ver cómo funcionan los sistemas y cómo interactúan entre sí"

# Probar el modelo con el texto de ejemplo
predicted_label_cargado = predict_with_loaded_model(loaded_model, tokenizer, texto_a_probar_cargado)

# Imprimir la etiqueta predicha de forma legible
print(f"Texto: {texto_a_probar_cargado}")
print(f"Etiqueta Predicha con modelo cargado: {predicted_label_cargado}")

Texto: Adoro ver cómo funcionan los sistemas y cómo interactúan entre sí
Etiqueta Predicha con modelo cargado: Ing. En Sistemas


In [19]:
# Ejemplo de texto para probar el modelo cargado
texto_a_probar_cargado = "Desde pequeño siempre me llamo la atencion la programacion y participe en eventos de programacion competitiva"

# Probar el modelo con el texto de ejemplo
predicted_label_cargado = predict_with_loaded_model(loaded_model, tokenizer, texto_a_probar_cargado)

# Imprimir la etiqueta predicha de forma legible
print(f"Texto: {texto_a_probar_cargado}")
print(f"Etiqueta Predicha con modelo cargado: {predicted_label_cargado}")

Texto: Desde pequeño siempre me llamo la atencion la programacion y participe en eventos de programacion competitiva
Etiqueta Predicha con modelo cargado: Ing. En Ciencias de la Computación


In [20]:
# Ejemplo de texto para probar el modelo cargado
texto_a_probar_cargado = "La producción de contenido visual siempre ha sido una pasion mia"

# Probar el modelo con el texto de ejemplo
predicted_label_cargado = predict_with_loaded_model(loaded_model, tokenizer, texto_a_probar_cargado)

# Imprimir la etiqueta predicha de forma legible
print(f"Texto: {texto_a_probar_cargado}")
print(f"Etiqueta Predicha con modelo cargado: {predicted_label_cargado}")

Texto: La producción de contenido visual siempre ha sido una pasion mia
Etiqueta Predicha con modelo cargado: Ing. En Diseño y Animacion Digital


In [21]:
# Ejemplo de texto para probar el modelo cargado
texto_a_probar_cargado = "Tengo interes en aprender sobre redes, ingenieria social y como se mueve la informacion"

# Probar el modelo con el texto de ejemplo
predicted_label_cargado = predict_with_loaded_model(loaded_model, tokenizer, texto_a_probar_cargado)

# Imprimir la etiqueta predicha de forma legible
print(f"Texto: {texto_a_probar_cargado}")
print(f"Etiqueta Predicha con modelo cargado: {predicted_label_cargado}")

Texto: Tengo interes en aprender sobre redes, ingenieria social y como se mueve la informacion
Etiqueta Predicha con modelo cargado: Ing. En Tecnologías de la Información y Seguridad


In [22]:
def predict_with_loaded_model(model, tokenizer, text):
    # Tokenizar el texto de entrada
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=model.tokenizer.model_max_length, return_tensors='pt', return_attention_mask=True)

    # Obtener predicciones del modelo cargado
    with torch.no_grad():
        logits = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])

    # Aplicar softmax para obtener las probabilidades
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Obtener la etiqueta predicha y el porcentaje de confianza máximo
    max_probability, predicted_class = torch.max(probabilities, 1)
    confidence_percentage = max_probability.item() * 100

    return label_mapping(predicted_class.item()), confidence_percentage

# Ejemplo de texto para probar el modelo cargado
texto_a_probar_cargado = "pintar"

# Probar el modelo con el texto de ejemplo
predicted_label_cargado, confidence_percentage = predict_with_loaded_model(loaded_model, tokenizer, texto_a_probar_cargado)

# Imprimir la etiqueta predicha y el porcentaje de confianza de forma legible
print(f"Texto: {texto_a_probar_cargado}")
print(f"Etiqueta Predicha con modelo cargado: {predicted_label_cargado}")
print(f"Confianza: {confidence_percentage:.2f}%")


Texto: pintar
Etiqueta Predicha con modelo cargado: Ing. En Sistemas
Confianza: 42.48%
